## Solve using Benders

In [1]:
from gurobipy import *

In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [3]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

In [4]:
N_CONDS = 2

In [5]:
conds = df_meas.substrate.unique()
ex_rxns = df_meas.ex_rxn.unique()

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [6]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import cobra

# Just need fitted model for its kopt
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')
# This is the basal model
ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

### Stack models
stacker = StackOptimizer()

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7faa81585e60>)


In [7]:
opt = Optimizer(ijomc)
%time gap = opt.add_duality_gap_constraint()
%time stacker.stack_models(gap, df_conds)

CPU times: user 13.8 s, sys: 87.1 ms, total: 13.8 s
Wall time: 13.9 s
CPU times: user 3.74 s, sys: 48.4 ms, total: 3.79 s
Wall time: 3.78 s


In [8]:
stacker.model.metabolites.query('duality')

[<Metabolite duality_gap_glucose at 0x7faa1b906810>,
 <Metabolite duality_gap_maltose at 0x7faa1e66e750>]

In [9]:
stacker.model.metabolites.query('crowding')

[<Metabolite crowding_glucose at 0x7faa1b8ea690>,
 <Metabolite crowding_maltose at 0x7faa1e64d5d0>]

In [10]:
crowding = ijomc.metabolites.crowding
crowdingfit = ijofit.metabolites.crowding
a12_dict = {(crowding.id, rxn.id):(
    rxn.metabolites[crowding],
    ijofit.reactions.get_by_id(rxn.id).metabolites[crowdingfit]) for rxn in crowding.reactions}

In [11]:
opt = Optimizer(stacker.model)
opt.stack_disjunctive(stacker.model, a12_dict, stacker.model_dict.keys())

<Model stacked at 0x7faa2107f9d0>

In [13]:
stacked_model = stacker.model

for rxn in stacked_model.reactions:
    rxn.objective_coefficient = 0.
for rxn in stacked_model.reactions.query('binary_'):
    rxn.objective_coefficient = 1.

In [22]:
### Set conditions and measurements
F_TOL = 0.05

mu_id = ijomc.reactions.BIOMASS_Ec_iJO1366_core_53p95M.id

for cond,mdl in iteritems(stacker.model_dict):    
    row = df_meas[ df_meas.substrate==cond].iloc[0]    
    ex_id = row['ex_rxn']
    mu_meas = row['growth_rate_1_h']
    
    # Set this bound
    muL = mu_meas*(1+F_TOL)
    muU = mu_meas*(1-F_TOL)
    
    print('Condition %s: constraining %g <= mu <= %g' % (cond, muL, muU))
    
    mu_rxn = stacked_model.reactions.get_by_id(mu_id+'_%s'%cond)    
    mu_rxn.upper_bound = muL
    mu_rxn.lower_bound = muU

Condition maltose: constraining 0.651 <= mu <= 0.589
Condition glucose: constraining 0.777 <= mu <= 0.703


In [24]:
[(r.id, r.lower_bound, r.upper_bound) for r in stacked_model.reactions.query('BIOMASS_Ec_iJO1366_core_')]

[(u'BIOMASS_Ec_iJO1366_core_53p95M_glucose', 0.703, 0.777),
 (u'wl_BIOMASS_Ec_iJO1366_core_53p95M_glucose', 0.0, 1000.0),
 (u'wu_BIOMASS_Ec_iJO1366_core_53p95M_glucose', 0.0, 1000.0),
 (u'BIOMASS_Ec_iJO1366_core_53p95M_maltose', 0.589, 0.651),
 (u'wl_BIOMASS_Ec_iJO1366_core_53p95M_maltose', 0.0, 1000.0),
 (u'wu_BIOMASS_Ec_iJO1366_core_53p95M_maltose', 0.0, 1000.0)]

In [ ]:
%%time
stacked_model.optimize(solver='gurobi', objective_sense='minimize')

In [ ]:
sum([r.x for r in stacked_model.reactions.query('binary_')])

In [ ]:
for rxn in stacked_model.reactions.query('binary_'):
    if rxn.x > 0.99:
        print rxn, rxn.x

## Solve as MILP

In [26]:
from cobra.solvers import gurobi_solver
from gurobipy import *

In [27]:
%%time
milp = gurobi_solver.create_problem(stacked_model)
milp.ModelSense = GRB.MINIMIZE

CPU times: user 1.2 s, sys: 51.2 ms, total: 1.25 s
Wall time: 1.22 s


In [28]:
milp.Params.IntFeasTol = 1e-9
# milp.Params.FeasibilityTol = 1e-8
# milp.Params.OptimalityTol = 1e-8
# milp.Params.Presolve = 0
milp.Params.NumericFocus = 1

In [ ]:
milp.Params.OutputFlag=1
%time milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 44566 rows, 35030 columns and 159086 nonzeros
Variable types: 33012 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-11, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [5e-03, 1e+04]
Presolve removed 15950 rows and 10917 columns
Presolve time: 0.70s
Presolved: 28616 rows, 24113 columns, 111436 nonzeros
Variable types: 22095 continuous, 2018 integer (2018 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   30537    1.4797590e+02   2.114500e-03   3.515468e+08      5s
   34087    2.3192890e-01   0.000000e+00   0.000000e+00      7s

Root relaxation: objective 2.319289e-01, 34087 iterations, 6.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [ ]:
milp.Status == GRB.OPTIMAL

In [ ]:
milp.ObjVal

In [99]:
sol = gurobi_solver.format_solution(milp, stacked_model)

In [101]:
for rxn in stacked_model.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        print rxn, rxn.x, xmilp

binary_crowding_ATPS4rpp_abs 1.0 1.0


## Solve using Benders with quad/dqq subproblem

In [28]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

decomposer = Decomposer(milp)
master,sub = decomposer.benders_decomp()
master._precision_sub = 'quad'
master._verbosity = 1

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [42]:
sub.qsolver.opt_realdict['lp']

{'Feasibility tol': 1e-15,
 'LU factor tol': 10.0,
 'LU singularity tol': 1e-30,
 'LU update tol': 10.0,
 'Optimality tol': 1e-15,
 'Penalty parameter': 100.0,
 'Unbounded step size': 1e+30}

In [29]:
%%time
master.optimize(cb_benders)

Optimize a model with 1 rows, 2019 columns and 2019 nonzeros
Variable types: 1 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Getting MINOS parameters...
Done in 225.226 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Getting MINOS parameters...
Done in 1.35714 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Getting MINOS parameters...
Done in 1.31344 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303
Variable types: 1 continuous, 2018 integer (2018 binary)
Getting MINOS parameters...
Done in 1.29678 seconds with status 0
zmaster=0. zsub=3.70303. gap=-3.70303

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.785988e-03   0.000000e+00    233s
       2    6.2219142e-02   0.000000e+00   0.000000e+00    233s

Root relaxation: objective 6.221914e-

In [30]:
import numpy as np
yopt = np.array([y.X for y in decomposer._ys])
sum(yopt)

1.0

In [31]:
y_dict = {y.VarName:y.X for y in decomposer._ys}

In [32]:
from qminos.quadLP import QMINOS
### SOlve primal sub
qsolver = QMINOS()
sense_dict = {GRB.EQUAL:'E', GRB.LESS_EQUAL:'L', GRB.GREATER_EQUAL:'G'}
csenses = [sense_dict[c] for c in decomposer._csenses]
xp,statp,hsp = qsolver.solvelp(decomposer._A, decomposer._d - decomposer._B*yopt, decomposer._cx, decomposer._xl, decomposer._xu, csenses, precision='dqq')

Getting MINOS parameters...
Done in 21.6104 seconds with status 0


In [33]:
x_dict = {x.VarName:xp[j] for j,x in enumerate(decomposer._x0)}

In [34]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7490032656265933

In [35]:
milp.getVars()[7]

<gurobi.Var 7 (value 0.749027199175)>

In [36]:
x_dict['7']

0.7490271991752928

In [37]:
print(gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound, gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound)

(0.6936745323599769, 0.7666929041873429)


In [38]:
rids = [r.id for r in gap.reactions]
yid_dict = {y.VarName:rids[int(y.VarName)] for y in decomposer._y0}
y_inds = {rids[int(y.VarName)]:y.VarName for y in decomposer._y0}

{u'binary_crowding_GLCURtex_abs': '12920',
 u'binary_crowding_SPODM': '15479',
 u'binary_crowding_THRS': '16496',
 u'binary_crowding_DINSt2pp': '17126',
 u'binary_crowding_THRD': '14618',
 u'binary_crowding_RPI_abs': '15749',
 u'binary_crowding_AMALT3': '16676',
 u'binary_crowding_DHPPD': '15404',
 u'binary_crowding_METSOX1abcpp': '12632',
 u'binary_crowding_DAPAL': '16184',
 u'binary_crowding_ACACCT': '15716',
 u'binary_crowding_MINCYCtpp': '14459',
 u'binary_crowding_G3PCabcpp': '17675',
 u'binary_crowding_AGM4PCPpp': '18017',
 u'binary_crowding_TTRCYCtex_abs': '13409',
 u'binary_crowding_PAPA140': '14714',
 u'binary_crowding_PAPA141': '13013',
 u'binary_crowding_TRSARr_abs': '12485',
 u'binary_crowding_FE3HOXexs': '13361',
 u'binary_crowding_GGGABAH': '18233',
 u'binary_crowding_GLYBt2pp': '16268',
 u'binary_crowding_PE120abcpp': '15227',
 u'binary_crowding_FUSAtpp': '17144',
 u'binary_crowding_GLCDpp': '17120',
 u'binary_crowding_PUNP2_abs': '17165',
 u'binary_crowding_FRUpts2pp': 

In [39]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        xbend = y_dict[y_inds[rxn.id]]
        print rxn, rxn.x, xmilp, xbend

binary_crowding_ATPS4rpp_abs 1.0 1.0 1.0
